In [16]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/' 
sys.path.append(parent_dir) 

import pandas as pd
import numpy as np
from copy import deepcopy

from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time, auto_save_pickle

from models.fmin2 import fmin2, fmin2_model1
from models.nested_validation import *
from models.features import fixed_makedata, salvador_wrap, pretrained_wrap
from models.display import pickle2df

# models
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

pd.set_option('display.max_colwidth', -1)

import warnings
warnings.filterwarnings("ignore")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.54 µs
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 41 ms


In [17]:
dir_save = os.path.normpath('../data/dic_20190819.pickle')
dic = load(dir_save)

amt_GI = load_pickle('../data/amt_hGI_pos.pickle')
for i, v in dic.items():
    if i in amt_GI.keys():
        v['AMT'] = amt_GI[i]
amt_ls = list(amt_GI.keys())
ls = amt_ls

exist
time: 20.8 s


### Nutritions only

In [3]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lr
    
    spaces = {'model1': model1, 'model2': LogisticRegression,'method': method,
              'p2': p2_lr, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 10)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT')
knowns = corp0.knowns
state = 2022
model1 = None
for method in [['nu','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-22 14:26:40.951796
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 0.5}
2019-08-22 14:26:41.168117
[0.7857142857142857, 0.8098159509202455, 0.8152866242038217, 0.8481012658227849, 0.8627450980392157]
0.8243326449400709
-0.8243326449400709 -0.8243326449400709
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}
2019-08-22 14:26:45.993830
[0.7857142857142857, 0.8024691358024691, 0.8152866242038217, 0.8481012658227849, 0.8684210526315789]
0.823998472834988
-0.823998472834988 -0.8243326449400709
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 3.0}
2019-08-22 14:26:50.303324
[0.7810650887573963, 0.8024691358024691, 0.8152866242038217, 0.8481012658227849, 0.8684210526315789]
0.8230686334436103
-0.8230686334436103 -0.8243326449400709
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1000}
2019-08-22 14:26:55.800620
[0.7810650887573963, 0.8074534161490683, 0.81528

test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.825   0.83           0.822       0.822         0.839    0.841            

  train_recall train_accuracy model1  \
0  0.837        0.836          None    

                                                                                                                                                                                                                                                                              model2  \
0  LogisticRegression(C=0.1, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

        method  tag                                  pickle_path  
0  [nu, scale]  AMT  ../pickle/2019-08-22 14:30:58.313921.pickle

save to ../csv/2019-08-22 14:30:58.433842.csv


test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.825   0.83           0.822       0.822         0.839    0.841            

  train_recall train_accuracy model1  \
0  0.837        0.836          None    

                                                                                                                                                                                                                                                                              model2  \
0  LogisticRegression(C=0.1, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

        method  tag                                  pickle_path  
0  [nu, scale]  AMT  ../pickle/2019-08-22 14:30:58.313921.pickle

time: 4min 29s


### Combination1: word2vec + nutritions

In [32]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lr
    
    spaces = {'model1': model1, 'model2': LogisticRegression,'method': method,
              'p2': p2_lr, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 10)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT')
knowns = corp0.knowns
state = 2022
model1 = None
for method in [['wc', 'nb', 'nu','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-22 10:48:56.881425
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 0.5}
2019-08-22 10:48:57.155400
[0.7901234567901234, 0.8242424242424242, 0.8461538461538461, 0.8048780487804877, 0.8079470198675496]
0.8146689591668862
-0.8146689591668862 -0.8146689591668862
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}
2019-08-22 10:49:11.974025
[0.7852760736196319, 0.8148148148148148, 0.830188679245283, 0.7975460122699387, 0.8133333333333334]
0.8082317826566003
-0.8082317826566003 -0.8146689591668862
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 3.0}
2019-08-22 10:49:25.918803
[0.7804878048780488, 0.8098159509202455, 0.830188679245283, 0.7926829268292682, 0.8211920529801324]
0.8068734829705955
-0.8068734829705955 -0.8146689591668862
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1000}
2019-08-22 10:49:40.881301
[0.7484662576687118, 0.8023952095808382, 0.80745

test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.85    0.859          0.842       0.847         0.933    0.937            

  train_recall train_accuracy model1  \
0  0.929        0.932          None    

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.05, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

                method  tag                                  pickle_path  
0  [wc, nb, nu, scale]  AMT  ../pickle/2019-08-22 11:00:45.957898.pickle

save to ../csv/2019-08-22 11:00:46.010297.csv


test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.85    0.859          0.842       0.847         0.933    0.937            

  train_recall train_accuracy model1  \
0  0.929        0.932          None    

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.05, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

                method  tag                                  pickle_path  
0  [wc, nb, nu, scale]  AMT  ../pickle/2019-08-22 11:00:45.957898.pickle

time: 12min 2s


### Combination2: Recipe1M word embedding and Nutritions

In [8]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lr
    
    spaces = {'model1': model1, 'model2': LogisticRegression,'method': method,
              'p2': p2_lr, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 10)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT')
knowns = corp0.knowns
state = 2022
model1 = salvador_wrap()
for method in [['wv','nu','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-22 12:49:55.149257
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 0.5}
2019-08-22 12:49:55.399637
[0.7948717948717949, 0.7798742138364779, 0.7928994082840236, 0.7901234567901234, 0.7891156462585034]
0.7893769040081847
-0.7893769040081847 -0.7893769040081847
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}
2019-08-22 12:50:31.045960
[0.8, 0.7975460122699387, 0.7784431137724551, 0.7975460122699387, 0.7724137931034483]
0.7891897862831561
-0.7891897862831561 -0.7893769040081847
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 3.0}
2019-08-22 12:51:02.984173
[0.7612903225806451, 0.7607361963190185, 0.7710843373493975, 0.7926829268292682, 0.7285714285714286]
0.7628730423299516
-0.7628730423299516 -0.7893769040081847
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1000}
2019-08-22 12:51:35.225145
[0.7564102564102564, 0.7317073170731708, 0.7544910179640719, 

test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.836   0.851          0.824       0.834         0.906    0.923            

  train_recall train_accuracy  \
0  0.889        0.906           

                                                     model1  \
0  <models.features.salvador_wrap object at 0x7f9e0555b8d0>   

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.01, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

            method  tag                                  pickle_path  
0  [wv, nu, scale]  AMT  ../pickle/2019-08-22 13:17:50.876077.pickle

save to ../csv/2019-08-22 13:17:51.399916.csv


test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.836   0.851          0.824       0.834         0.906    0.923            

  train_recall train_accuracy  \
0  0.889        0.906           

                                                     model1  \
0  <models.features.salvador_wrap object at 0x7f9e0555b8d0>   

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.01, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

            method  tag                                  pickle_path  
0  [wv, nu, scale]  AMT  ../pickle/2019-08-22 13:17:50.876077.pickle

time: 28min 8s


### Combinations: skipthoughts + nutritons

In [18]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lr
    
    spaces = {'model1': model1, 'model2': LogisticRegression,'method': method,
              'p2': p2_lr, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 10)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT', add_skip_thoughts='../data/X_skipthoughts_25k.pickle')
knowns = corp0.knowns
state = 2022
model1 = None
for method in [['skip','nu','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-23 16:38:37.499084
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 0.5}
2019-08-23 16:38:37.773124
[0.7777777777777778, 0.7664670658682634, 0.75, 0.7757575757575756, 0.8375]
0.7815004838807232
-0.7815004838807232 -0.7815004838807232
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}
2019-08-23 16:38:46.620291
[0.7701863354037267, 0.7590361445783134, 0.7594936708860759, 0.7710843373493975, 0.8343558282208589]
0.7788312632876745
-0.7788312632876745 -0.7815004838807232
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 3.0}
2019-08-23 16:38:55.372599
[0.7701863354037267, 0.7544910179640719, 0.7643312101910827, 0.7710843373493975, 0.8220858895705521]
0.7764357580957661
-0.7764357580957661 -0.7815004838807232
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1000}
2019-08-23 16:39:03.993436
[0.7594936708860759, 0.738095238095238, 0.7515923566878981, 0.7590361445

test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.821   0.857          0.79        0.824         0.931    0.956            

  train_recall train_accuracy model1  \
0  0.908        0.932          None    

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.05, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

              method  tag                                  pickle_path  
0  [skip, nu, scale]  AMT  ../pickle/2019-08-23 16:46:04.479503.pickle

save to ../csv/2019-08-23 16:46:04.623991.csv


test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.821   0.857          0.79        0.824         0.931    0.956            

  train_recall train_accuracy model1  \
0  0.908        0.932          None    

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.05, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

              method  tag                                  pickle_path  
0  [skip, nu, scale]  AMT  ../pickle/2019-08-23 16:46:04.479503.pickle

time: 7min 40s


### combination: NB-BoW word2vec Nutritions

In [10]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lr
    
    spaces = {'model1': model1, 'model2': LogisticRegression,'method': method,
              'p2': p2_lr, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 10)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT')
knowns = corp0.knowns
state = 2022
model1 = salvador_wrap()
for method in [['wc','nb','wv','nu','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-22 13:23:41.857154
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 0.5}
2019-08-22 13:23:42.069352
[0.8227848101265822, 0.7926829268292682, 0.8023952095808382, 0.8433734939759034, 0.8079470198675496]
0.8138366920760284
-0.8138366920760284 -0.8138366920760284
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}
2019-08-22 13:24:18.525362
[0.8125000000000001, 0.787878787878788, 0.8048780487804877, 0.8433734939759034, 0.8108108108108107]
0.8118882282891979
-0.8118882282891979 -0.8138366920760284
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 3.0}
2019-08-22 13:24:56.100181
[0.810126582278481, 0.787878787878788, 0.7904191616766467, 0.8433734939759034, 0.794701986754967]
0.8053000025129572
-0.8053000025129572 -0.8138366920760284
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1000}
2019-08-22 13:25:32.937189
[0.7654320987654321, 0.7810650887573963, 0.7784431

test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.849   0.861          0.842       0.847         0.933    0.939            

  train_recall train_accuracy  \
0  0.928        0.933           

                                                     model1  \
0  <models.features.salvador_wrap object at 0x7f9ef020b3c8>   

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.01, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

                    method  tag                                  pickle_path  
0  [wc, nb, wv, nu, scale]  AMT  ../pickle/2019-08-22 13:58:57.413917.pickle

save to ../csv/2019-08-22 13:58:58.023957.csv


test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.849   0.861          0.842       0.847         0.933    0.939            

  train_recall train_accuracy  \
0  0.928        0.933           

                                                     model1  \
0  <models.features.salvador_wrap object at 0x7f9ef020b3c8>   

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.01, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

                    method  tag                                  pickle_path  
0  [wc, nb, wv, nu, scale]  AMT  ../pickle/2019-08-22 13:58:57.413917.pickle

time: 35min 27s


### everything

In [20]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lr
    
    spaces = {'model1': model1, 'model2': LogisticRegression,'method': method,
              'p2': p2_lr, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 10)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2


'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT', add_skip_thoughts='../data/X_skipthoughts_25k.pickle')
knowns = corp0.knowns
state = 2022
model1 = salvador_wrap()
for method in [['wc','nb','wv','nu','skip','scale']]:
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    print('state', state)
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2) ,method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-23 16:57:16.679679
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 0.5}
2019-08-23 16:57:16.958524
[0.8047337278106509, 0.7560975609756098, 0.7701863354037267, 0.8292682926829269, 0.8025477707006369]
0.7925667375147103
-0.7925667375147103 -0.7925667375147103
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}
2019-08-23 16:58:08.376041
[0.7976190476190476, 0.7636363636363638, 0.7654320987654321, 0.8214285714285715, 0.8025477707006369]
0.7901327704300104
-0.7901327704300104 -0.7925667375147103
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 3.0}
2019-08-23 16:58:59.930365
[0.7976190476190476, 0.7590361445783134, 0.75, 0.8263473053892216, 0.7974683544303798]
0.7860941704033925
-0.7860941704033925 -0.7925667375147103
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1000}
2019-08-23 16:59:52.102435
[0.7701863354037267, 0.757396449704142, 0.7295597484276728, 

test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.842   0.86           0.826       0.841         0.953    0.964            

  train_recall train_accuracy  \
0  0.942        0.953           

                                                     model1  \
0  <models.features.salvador_wrap object at 0x7f9e07f8c0b8>   

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.01, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

                          method  tag  \
0  [wc, nb, wv, nu, skip, scale]  AMT   

                                   pickle_path  
0  ../pickle/2019-08-23 17:41:26.141749.pickle

save to ../csv/2019-08-23 17:41:26.704592.csv


test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.842   0.86           0.826       0.841         0.953    0.964            

  train_recall train_accuracy  \
0  0.942        0.953           

                                                     model1  \
0  <models.features.salvador_wrap object at 0x7f9e07f8c0b8>   

                                                                                                                                                                                                                                                                               model2  \
0  LogisticRegression(C=0.01, class_weight='balanced', dual=False,\n          fit_intercept=True, intercept_scaling=1, max_iter=100,\n          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)   

                          method  tag  \
0  [wc, nb, wv, nu, skip, scale]  AMT   

                                   pickle_path  
0  ../pickle/2019-08-23 17:41:26.141749.pickle

time: 44min 24s
